In [59]:
import pandas as pd
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from operator import itemgetter

The goal of this task is to predict the "StarPlayerBrawler" of the WinningTeam in the mode "BrawlBall". 

1. We will read out the winning team (The Starplayer can only come from the Winning Team). 
2. The Attributes "WinningTeam", "map", the 3 "Winners" and the "StarPlayerBrawler" of the match should be visualised as a table.
3. We will create a model to predict the "StarPlayerBrawler" out of the 3 players of the WInningTeam

# Reading the Dataset

In [5]:
df = pd.read_csv("rawdata.csv")
df.head(5)

,WinningTeam,BattleTime,playerTag,mode,map,result,StarPlayerTag,StarPlayerBrawler,Brawler1Name,Brawler1Tag,Brawler2Name,Brawler2Tag,Brawler3Name,Brawler3Tag,Brawler4Name,Brawler4Tag,Brawler5Name,Brawler5Tag,Brawler6Name,Brawler6Tag
0,1,20200530T122433.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,BEA,#222VJU0CU,BROCK,#20C2RRRGY,EMZ,#Y220JJJ9
1,1,20200530T121808.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,GENE,#2R9YVLQY,MAX,#22L8CLYQU,PENNY,#28U009JPY
2,1,20200530T121207.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#9U2QJ0P,BEA,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,GENE,#8YVQRQP,BROCK,#9QJP9LP,SPIKE,#98VJCL2PQ
3,1,20200530T120359.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,JACKY,#2UPURLUG2,FRANK,#2UCV2R0LV,POCO,#22RURYRV0
4,1,20200530T120028.000Z,#9U2QJ0P,brawlBall,Backyard Bowl,1,#VGR0CJ29,PIPER,PIPER,#VGR0CJ29,BEA,#9U2QJ0P,GENE,#9UR09Q80Y,PIPER,#PUG8CR80,BROCK,#98JQVGLP9,JACKY,#8822ULJ9Q


# Appending Winners

Since Brawlball is 3 vs 3 mode, we only need the Winners. For each row, i will append the BrawlerNames only of the Winner group:

In [115]:
WinnerTeam = []

for index, row in df.iterrows():
    winners = []
    
    if row["WinningTeam"] == 1 and row["result"] != 0:
        winners.append(row["Brawler1Name"])
        winners.append(row["Brawler2Name"])
        winners.append(row["Brawler3Name"])
        
    
    elif row["result"] != 0:
        winners.append(row["Brawler4Name"])
        winners.append(row["Brawler5Name"])
        winners.append(row["Brawler6Name"])
    
    if len(winners) != 0:
        winners = sorted(winners)
        WinnerTeam.append(winners)

combinedWinnerTeam = []
    
for i in range(len(WinnerTeam)):
    winners = ""
    winners = winners + WinnerTeam[i][0]
    winners = winners + WinnerTeam[i][1]
    winners = winners + WinnerTeam[i][2]
    combinedWinnerTeam.append(winners)
    
print(dfFinal.shape)

(1113, 513)


# Creating Table (WinningTeam, map, 3 Winners, Starplayer)

In [120]:
dfFinal = pd.DataFrame(columns=["WinnerTeam", "map"])
dfFinal["WinnerTeam"] = combinedWinnerTeam
dfFinal["map"] =df["map"]
dfFinal["StarPlayerBrawler"] = df["StarPlayerBrawler"]

print(dfFinal.head(15))
print(dfFinal["WinnerTeam"].nunique())
print(dfFinal["map"].nunique())
target = dfFinal.loc[:,"StarPlayerBrawler"].copy()
dfFinal = pd.get_dummies(dfFinal.loc[:,["WinnerTeam", "map"]], dtype = "uint8")

dfFinal["StarPlayerBrawler"] = target

dfFinal[["WinnerTeam_BEAGENEPIPER", "map_Backyard Bowl", "StarPlayerBrawler"]].head(10)

        WinnerTeam             map StarPlayerBrawler
0     BEAGENEPIPER   Backyard Bowl               BEA
1     BEAGENEPIPER   Backyard Bowl               BEA
2     BEAGENEPIPER   Backyard Bowl               BEA
3     BEAGENEPIPER   Backyard Bowl             PIPER
4     BEAGENEPIPER   Backyard Bowl             PIPER
5     BEAGENEPIPER   Backyard Bowl              GENE
6     BEAGENEPIPER   Backyard Bowl             PIPER
7     BEAGENEPIPER   Backyard Bowl             PIPER
8     BEAGENEPIPER   Backyard Bowl              GENE
9   GENESANDYSPIKE  Triple Dribble             SANDY
10     MAXPAMPIPER  Triple Dribble               BEA
11   BEABROCKMR. P   Backyard Bowl               PAM
12   BEAMR. PSPIKE   Backyard Bowl               BEA
13   BEABROCKMR. P   Backyard Bowl             MR. P
14   BEABROCKMR. P   Backyard Bowl             MR. P
498
14


,WinnerTeam_BEAGENEPIPER,map_Backyard Bowl,StarPlayerBrawler
0,1,1,BEA
1,1,1,BEA
2,1,1,BEA
3,1,1,PIPER
4,1,1,PIPER
5,1,1,GENE
6,1,1,PIPER
7,1,1,PIPER
8,1,1,GENE
9,0,0,SANDY


# Creating Model to somehow predict, who's going to be StarPlayer out of the 3 winning players (maybe with stats like accurays, recall and precision?) 

In [104]:
x_features = dfFinal.loc[:,[x for x in dfFinal.columns if x.startswith("WinnerTeam") or x.startswith("map")]]

In [121]:
x_features = dfFinal.loc[:,[x for x in dfFinal.columns if x.startswith("WinnerTeam") or x.startswith("map")]]
print(x_features.shape)
winners = dfFinal.loc[:,["StarPlayerBrawler"]].values.reshape(-1,)

print(dfFinal.loc[:,"StarPlayerBrawler"].value_counts())

(1113, 512)
PAM         128
BEA         124
BROCK        87
EMZ          61
SANDY        53
PIPER        51
MORTIS       46
PENNY        45
SPROUT       44
COLT         44
POCO         42
BIBI         39
GALE         32
JACKY        28
BO           27
MAX          27
MR. P        25
CROW         25
GENE         24
SPIKE        21
FRANK        19
CARL         17
NITA         15
JESSIE       15
TARA         15
RICO         15
8-BIT        10
EL PRIMO      8
DARRYL        7
LEON          5
BULL          5
ROSA          4
DYNAMIKE      3
SHELLY        2
Name: StarPlayerBrawler, dtype: int64


In [122]:
tree1 = tree.DecisionTreeClassifier()
tree1 = tree1.fit(x_features, winners)

In [123]:
tree2 = RandomForestClassifier()
tree2 = tree2.fit(x_features, winners)

In [124]:
scores = cross_val_score(tree2, x_features, winners, scoring='accuracy')
np.mean(scores)

C:\Users\Sajan\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


0.12671999353613703

Shelly only twice in dataset --> which is less than 5 splits 